<a href="https://colab.research.google.com/github/emerson1000/notebooks/blob/main/Optimizacion_sistema_espera_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Optimización de parámetros en sistemas de espera**



**Utilizaremos un ejemplo de sistema MMC.**

Aquí, se utilizará el módulo scipy.optimize de la biblioteca SciPy para realizar una búsqueda de parámetros óptimos en el sistema MMC. El objetivo será minimizar el tiempo promedio de espera en la cola. Tendremos como variables a optimizar el número de servidores (entre 1 y 20) y la capacidad de espera (entre 0 y 100).

In [ ]:
pip install simpy

In [ ]:
pip install scipy

In [ ]:
import simpy
import random

# Parámetros del sistema
llegada_media = 7  # Tasa media de llegada de clientes (clientes por minuto)
servicio_media = 5  # Tasa media de servicio (clientes por minuto)
tiempo_simulacion = 480  # Tiempo total de simulación en minutos

# Listas para registrar tiempos de llegada y tiempos de inicio de servicio
tiempos_llegada = []
tiempos_inicio_servicio = []

# Función para simular el proceso de llegada de clientes
def llegada_clientes(env, servidor):
    while True:
        yield env.timeout(random.expovariate(1/llegada_media))
        tiempos_llegada.append(env.now)
        env.process(servicio_cliente(env, servidor))

# Función para simular el proceso de servicio a los clientes
def servicio_cliente(env, servidor):
    with servidor.request() as req:
        yield req
        tiempos_inicio_servicio.append(env.now)
        yield env.timeout(random.expovariate(1/servicio_media))

# Configuración de la simulación
env = simpy.Environment()
servidor = simpy.Resource(env, capacity=2)  # Servidores
env.process(llegada_clientes(env, servidor))

# Ejecutar la simulación
env.run(until=tiempo_simulacion)

# Calcular métricas de rendimiento
tiempo_promedio_espera = sum(tiempo_inicio - tiempo_llegada for tiempo_inicio, tiempo_llegada in zip(tiempos_inicio_servicio, tiempos_llegada)) / len(tiempos_llegada)
longitud_promedio_cola = tiempo_promedio_espera * llegada_media
print(f"Tiempo promedio de espera: {tiempo_promedio_espera:.2f} minutos")
print(f"Longitud promedio de la cola: {longitud_promedio_cola:.2f} clientes")



Tiempo promedio de espera: 0.20 minutos
Longitud promedio de la cola: 1.41 clientes


**Interpretación**

Tiempo Promedio de Espera: El tiempo promedio de espera es de aproximadamente 0.43 minutos. Esto significa que, en promedio, un cliente espera alrededor de 0.43 minutos antes de ser atendido por un servidor. Cuanto menor sea este valor, mejor, ya que indica tiempos de espera más cortos para los clientes.

Longitud Promedio de la Cola: La longitud promedio de la cola es de aproximadamente 2.98 clientes. Esto significa que, en promedio, hay alrededor de 2.98 clientes esperando en la cola en un momento dado. Cuanto menor sea este valor, menor será la congestión en la cola y, en general, será un sistema más eficiente.

In [ ]:
import simpy
import random
import numpy as np
from scipy.optimize import minimize

# Parámetros del sistema
llegada_media = 7  # Tasa media de llegada de clientes (clientes por minuto)
tiempo_simulacion = 480  # Tiempo total de simulación en minutos
max_servidores = 2  # Número máximo de servidores
max_capacidad_cola = 10  # Capacidad máxima de la cola

# Función para simular el sistema y calcular los costos
def simular_sistema(num_servidores, capacidad_cola):
    env = simpy.Environment()
    servidor = simpy.Resource(env, capacity=num_servidores)
    tiempos_llegada = []
    tiempos_inicio_servicio = []

    def llegada_clientes(env, servidor):
        while True:
            yield env.timeout(random.expovariate(1/llegada_media))
            tiempos_llegada.append(env.now)
            env.process(servicio_cliente(env, servidor))

    def servicio_cliente(env, servidor):
        with servidor.request() as req:
            yield req
            tiempos_inicio_servicio.append(env.now)
            yield env.timeout(random.expovariate(1/servicio_media))

    env.process(llegada_clientes(env, servidor))
    env.run(until=tiempo_simulacion)

    if len(tiempos_inicio_servicio) != len(tiempos_llegada):
        return None  # La simulación no se completó correctamente

    tiempo_promedio_espera = np.mean(np.array(tiempos_inicio_servicio) - np.array(tiempos_llegada))
    longitud_promedio_cola = tiempo_promedio_espera * llegada_media

    # Calcular costos
    costos_operacion = num_servidores * tiempo_promedio_espera
    costos_espera = longitud_promedio_cola * tiempo_promedio_espera

    # Costo total: una combinación ponderada de costos de operación y costos de espera
    costo_total = 0.7 * costos_operacion + 0.3 * costos_espera

    return costo_total

# Función de optimización para encontrar la combinación óptima de servidores y capacidad de espera
def optimizar_sistema(params):
    num_servidores, capacidad_cola = params
    costo = simular_sistema(num_servidores, capacidad_cola)
    if costo is None:
        # Penalizar configuraciones que no completaron la simulación
        return 1e6  # Un valor grande para penalizar
    return costo

# Optimización utilizando minimize
resultado = minimize(optimizar_sistema, [1, 1], bounds=[(1, max_servidores), (0, max_capacidad_cola)], method='L-BFGS-B')

# Resultados óptimos
num_servidores_optimo, capacidad_cola_optima = resultado.x
costo_optimo = resultado.fun

print(f"Número óptimo de servidores: {int(num_servidores_optimo)}")
print(f"Capacidad óptima de la cola: {int(capacidad_cola_optima)}")
print(f"Costo óptimo: ${costo_optimo:.2f}")


Número óptimo de servidores: 1
Capacidad óptima de la cola: 0
Costo óptimo: $0.70


**Interpretación**

Número óptimo de servidores: 1: Según la optimización, el número óptimo de servidores en el sistema de colas es 1. Esto significa que, según los criterios utilizados en la optimización (en este caso, minimizar los costos totales), se determinó que un solo servidor es la configuración óptima.

Capacidad óptima de la cola: 1: La capacidad óptima de la cola es de 1. Esto indica que se ha encontrado que mantener solo un cliente en la cola es la configuración óptima. En otras palabras, cuando un servidor está ocupado y un cliente adicional llega, solo se permite que un cliente espere en la cola. Los clientes adicionales que lleguen mientras la cola esté llena pueden ser rechazados o tratados de manera diferente según las reglas del sistema.

Costo óptimo: 1000000.00: El costo óptimo calculado en esta configuración es de 1,000,000.00. Esto es una cifra bastante alta y podría indicar que, en esta configuración, se están incurriendo en costos significativos debido a tiempos de espera prolongados o ineficiencias en el sistema. Sin embargo, es importante recordar que los valores de costo son relativos y dependen de la ponderación de los costos de operación y los costos de espera en la función de costo utilizada en la optimización.

## **Costos de espera**

Supongamos que el costo de espera por minuto por cliente es de 1 (unidad monetaria). Si el tiempo promedio de espera es de 5 minutos y tienes 10 clientes esperando, el costo de espera sería de $50.

In [ ]:
costo_por_minuto_por_cliente = 1
tiempo_promedio_espera = 5  # minutos
cantidad_clientes_en_espera = 10
costo_espera_total = costo_por_minuto_por_cliente * tiempo_promedio_espera * cantidad_clientes_en_espera
print(f"Costo total de espera: ${int(costo_espera_total)}")

Costo total de espera: $50


### Costo de espera con simulación

In [ ]:
import simpy
import random

# Parámetros del sistema
llegada_media = 5  # Tasa media de llegada de clientes (clientes por minuto)
servicio_media = 3  # Tasa media de servicio (clientes por minuto)
tiempo_simulacion = 480  # Tiempo total de simulación en minutos

# Listas para registrar tiempos de llegada y tiempos de inicio de servicio
tiempos_llegada = []
tiempos_inicio_servicio = []

# Costo de espera por minuto (reemplaza con tu valor)
costo_espera_por_minuto = 10  # Ejemplo: $10 por minuto

# Función para simular el proceso de llegada de clientes
def llegada_clientes(env, servidor):
    while True:
        yield env.timeout(random.expovariate(1/llegada_media))
        tiempos_llegada.append(env.now)
        env.process(servicio_cliente(env, servidor))

# Función para simular el proceso de servicio a los clientes
def servicio_cliente(env, servidor):
    with servidor.request() as req:
        yield req
        tiempos_inicio_servicio.append(env.now)
        yield env.timeout(random.expovariate(1/servicio_media))

# Configuración de la simulación
env = simpy.Environment()
servidor = simpy.Resource(env, capacity=1)  # Servidor único
env.process(llegada_clientes(env, servidor))

# Ejecutar la simulación
env.run(until=tiempo_simulacion)

# Calcular métricas de rendimiento
tiempo_promedio_espera = sum(tiempo_inicio - tiempo_llegada for tiempo_inicio, tiempo_llegada in zip(tiempos_inicio_servicio, tiempos_llegada)) / len(tiempos_llegada)
longitud_promedio_cola = tiempo_promedio_espera * llegada_media

# Calcular costos de espera
costo_espera = tiempo_promedio_espera * costo_espera_por_minuto

print(f"Tiempo promedio de espera: {tiempo_promedio_espera:.2f} minutos")
print(f"Longitud promedio de la cola: {longitud_promedio_cola:.2f} clientes")
print(f"Costo de espera: ${costo_espera:.2f}")


Tiempo promedio de espera: 3.69 minutos
Longitud promedio de la cola: 18.45 clientes
Costo de espera: $36.91


## **Costo de linea o servidores**

Supongamos que el costo de operar un servidor es de 50 (um) por hora y tienes 2 servidores trabajando durante 8 horas al día. El costo de la línea sería de $800 por día.

In [ ]:
costo_por_servidor_por_hora = 50
numero_de_servidores = 2
horas_por_dia = 8
costo_linea_total = costo_por_servidor_por_hora * numero_de_servidores * horas_por_dia
print(f"Costo de línea total: ${int(costo_linea_total)}")

Costo de línea total: $800


El equilibrio entre estos dos costos se encuentra ajustando la cantidad de servidores y la capacidad de la cola. Un aumento en el número de servidores generalmente reduce el costo de espera, pero aumenta el costo de la línea. Por otro lado, una mayor capacidad de la cola puede reducir el costo de espera al permitir que más clientes esperen, pero también puede aumentar los costos de operación.

### Costo de servidores mediante simulación

In [ ]:
import simpy
import random

# Parámetros del sistema
llegada_media = 5  # Tasa media de llegada de clientes (clientes por minuto)
servicio_media = 6  # Tasa media de servicio (clientes por minuto)
tiempo_simulacion = 480  # Tiempo total de simulación en minutos

# Costo de operación por servidor por minuto (reemplaza con tu valor)
costo_operacion_por_minuto = 20  # Ejemplo: $20 por minuto por servidor

# Función para simular el sistema y calcular los costos
def simular_sistema(num_servidores):
    env = simpy.Environment()
    servidor = simpy.Resource(env, capacity=num_servidores)
    tiempos_llegada = []
    tiempos_inicio_servicio = []

    def llegada_clientes(env, servidor):
        while True:
            yield env.timeout(random.expovariate(1/llegada_media))
            tiempos_llegada.append(env.now)
            env.process(servicio_cliente(env, servidor))

    def servicio_cliente(env, servidor):
        with servidor.request() as req:
            yield req
            tiempos_inicio_servicio.append(env.now)
            yield env.timeout(random.expovariate(1/servicio_media))

    env.process(llegada_clientes(env, servidor))
    env.run(until=tiempo_simulacion)

    tiempo_promedio_espera = sum(tiempo_inicio - tiempo_llegada for tiempo_inicio, tiempo_llegada in zip(tiempos_inicio_servicio, tiempos_llegada)) / len(tiempos_llegada)

    # Costo de operación
    costo_operacion = num_servidores * costo_operacion_por_minuto * tiempo_simulacion

    return costo_operacion

# Algoritmo para encontrar el número óptimo de servidores
num_servidores_optimo = None
costo_optimo = float('inf')  # Inicializar con un valor alto

for num_servidores in range(1, 11):  # Prueba con diferentes números de servidores (de 1 a 10)
    costo = simular_sistema(num_servidores)
    if costo < costo_optimo:
        costo_optimo = costo
        num_servidores_optimo = num_servidores

print(f"Número óptimo de servidores: {num_servidores_optimo}")
print(f"Costo óptimo de línea: ${costo_optimo:.2f}")

Número óptimo de servidores: 1
Costo óptimo de línea: $9600.00


## **Capacidad de la cola**

Supongamos que estás diseñando un sistema de cola para un centro de llamadas y que la capacidad de la cola es de 20 clientes. Si en un momento dado hay 25 clientes esperando, los cinco clientes adicionales se considerarán "perdidos" o "rechazados" debido a que la capacidad de la cola se ha alcanzado.

In [ ]:
capacidad_de_cola = 20
clientes_en_espera = 25

if clientes_en_espera > capacidad_de_cola:
    clientes_perdidos = clientes_en_espera - capacidad_de_cola
    print(f"{clientes_perdidos} clientes fueron rechazados debido a que la capacidad de la cola se alcanzó.")


5 clientes fueron rechazados debido a que la capacidad de la cola se alcanzó.


La capacidad de la cola es un parámetro crítico que debe ajustarse cuidadosamente en función de la demanda prevista y los recursos disponibles. Un aumento en la capacidad de la cola puede reducir la tasa de rechazo de clientes, pero también puede aumentar los costos operativos debido a una cola más larga y la necesidad de recursos adicionales.

### Capacidad de la cola usando simulación

In [ ]:
import simpy
import random

# Parámetros del sistema
llegada_media = 5  # Tasa media de llegada de clientes (clientes por minuto)
servicio_media = 6  # Tasa media de servicio (clientes por minuto)
tiempo_simulacion = 480  # Tiempo total de simulación en minutos

# Costo de operación por servidor por minuto (reemplaza con tu valor)
costo_operacion_por_minuto = 20  # Ejemplo: $20 por minuto por servidor

# Función para simular el sistema y calcular los costos
def simular_sistema(capacidad_cola):
    env = simpy.Environment()
    servidor = simpy.Resource(env, capacity=1)  # Servidor único
    cola = simpy.Store(env, capacity=capacidad_cola)
    tiempos_llegada = []
    tiempos_inicio_servicio = []

    def llegada_clientes(env, servidor, cola):
        while True:
            yield env.timeout(random.expovariate(1/llegada_media))
            tiempos_llegada.append(env.now)
            env.process(servicio_cliente(env, servidor, cola))

    def servicio_cliente(env, servidor, cola):
        with servidor.request() as req:
            yield req
            tiempos_inicio_servicio.append(env.now)
            yield env.timeout(random.expovariate(1/servicio_media))

    def atender_cola(env, servidor, cola):
        while True:
            if len(cola.items) > 0:
                cliente = cola.get()
                env.process(servicio_cliente(env, servidor, cola))
            yield env.timeout(1)  # Verificar la cola cada minuto

    env.process(llegada_clientes(env, servidor, cola))
    env.process(atender_cola(env, servidor, cola))
    env.run(until=tiempo_simulacion)

    tiempo_promedio_espera = sum(tiempo_inicio - tiempo_llegada for tiempo_inicio, tiempo_llegada in zip(tiempos_inicio_servicio, tiempos_llegada)) / len(tiempos_llegada)

    # Costo de operación
    costo_operacion = costo_operacion_por_minuto * tiempo_simulacion

    return costo_operacion

# Algoritmo para encontrar la capacidad óptima de la cola
capacidad_cola_optima = None
costo_optimo = float('inf')  # Inicializar con un valor alto

for capacidad_cola in range(1, 11):  # Prueba con diferentes capacidades de cola (de 1 a 10)
    costo = simular_sistema(capacidad_cola)
    if costo < costo_optimo:
        costo_optimo = costo
        capacidad_cola_optima = capacidad_cola

print(f"Capacidad óptima de la cola: {capacidad_cola_optima}")
print(f"Costo óptimo de línea: ${costo_optimo:.2f}")


Capacidad óptima de la cola: 1
Costo óptimo de línea: $9600.00


##**Simulaciones**

Supongamos que tienes una fábrica que produce productos en una línea de ensamblaje y estás tratando de optimizar el sistema de cola para minimizar los costos. Vamos a considerar estos conceptos en un escenario industrial:

**Ejemplo de Costo de Espera y Capacidad de la Cola en una Fábrica:**

En una fábrica, los productos pasan por una línea de ensamblaje y pueden requerir tiempo de espera antes de ser procesados por máquinas o trabajadores. El costo de espera se refiere al costo de tener productos esperando en la cola, lo que puede incluir costos de almacenamiento, depreciación de productos y oportunidad perdida de producción. La capacidad de la cola es la cantidad máxima de productos que la fábrica puede tener en espera antes de que sea necesario detener la producción.

In [ ]:
import simpy
import random

# Parámetros del sistema
tiempo_produccion = 5  # Tiempo promedio para producir un producto (en minutos)
capacidad_de_cola = 10  # Capacidad máxima de la cola en la línea de ensamblaje

# Función para simular la línea de ensamblaje
def linea_de_ensamblaje(env, capacidad_cola):
    cola = simpy.Resource(env, capacity=capacidad_cola)
    while True:
        yield env.timeout(random.expovariate(1/tiempo_produccion))
        with cola.request() as req:
            yield req
            yield env.timeout(random.uniform(1, 2))  # Tiempo de procesamiento
            print(f"Producto ensamblado en t={env.now}")

# Configuración de la simulación
env = simpy.Environment()
env.process(linea_de_ensamblaje(env, capacidad_de_cola))

# Ejecutar la simulación
env.run(until=50)  # Simulación de 50 minutos


Producto ensamblado en t=2.6734171843223966
Producto ensamblado en t=4.387132648404866
Producto ensamblado en t=10.617404100146706
Producto ensamblado en t=12.23431096552767
Producto ensamblado en t=19.197552599182803
Producto ensamblado en t=25.83836840504629
Producto ensamblado en t=32.33820396327601
Producto ensamblado en t=34.2838911044825
Producto ensamblado en t=39.67553888621917
Producto ensamblado en t=44.31850123534311


En este ejemplo, estamos simulando una línea de ensamblaje en una fábrica donde se producen productos. La capacidad de la cola es de 10 productos. Puedes ajustar la capacidad de la cola para ver cómo afecta al costo de espera y a la eficiencia de la producción.

**Ejemplo de Costo de la Línea en una Estación de Procesamiento:**

Supongamos que tienes una estación de procesamiento de datos en un centro de cómputo y estás tratando de optimizar el número de servidores para minimizar los costos de operación.

In [ ]:
import simpy
import random

# Parámetros del sistema
llegada_media = 4  # Tasa media de llegada de trabajos (trabajos por minuto)
tiempo_simulacion = 48  # Tiempo total de simulación en minutos

# Función para simular la estación de procesamiento y calcular costos
def simular_estacion(env, num_servidores, resultados):
    servidor = simpy.Resource(env, capacity=num_servidores)
    trabajos_procesados = 0
    tiempo_total_espera = 0
    costo_operacion = 0

    while True:
        yield env.timeout(random.expovariate(1/llegada_media))
        llegada_trabajo = env.now

        with servidor.request() as req:
            yield req
            yield env.timeout(random.uniform(1, 2))  # Tiempo de procesamiento
            tiempo_espera = env.now - llegada_trabajo
            tiempo_total_espera += tiempo_espera
            trabajos_procesados += 1
            costo_operacion += num_servidores * tiempo_espera  # Costo de operación

            print(f"Trabajo procesado en t={env.now}, tiempo de espera: {tiempo_espera:.2f} minutos")

    tiempo_promedio_espera = tiempo_total_espera / trabajos_procesados
    resultados[num_servidores] = (tiempo_promedio_espera, costo_operacion)

# Configuración de la simulación
env = simpy.Environment()

num_servidores_valores = [1, 2, 3, 4, 5]  # Diferentes cantidades de servidores
resultados = {}  # Diccionario para almacenar los resultados

# Simulación con diferentes cantidades de servidores
for num_servidores in num_servidores_valores:
    env.process(simular_estacion(env, num_servidores, resultados))

# Ejecutar la simulación
env.run(until=tiempo_simulacion)

print(f"Cantidad de servidores: {num_servidores}")

Trabajo procesado en t=3.558420085125596, tiempo de espera: 1.31 minutos
Trabajo procesado en t=3.9661962732667493, tiempo de espera: 1.94 minutos
Trabajo procesado en t=4.744882653753985, tiempo de espera: 1.80 minutos
Trabajo procesado en t=6.145435536570523, tiempo de espera: 1.91 minutos
Trabajo procesado en t=6.253928240671851, tiempo de espera: 1.75 minutos
Trabajo procesado en t=6.952291275228239, tiempo de espera: 1.18 minutos
Trabajo procesado en t=7.509572027684142, tiempo de espera: 1.41 minutos
Trabajo procesado en t=9.091504109813854, tiempo de espera: 1.80 minutos
Trabajo procesado en t=10.71575182692498, tiempo de espera: 1.09 minutos
Trabajo procesado en t=11.694224386322615, tiempo de espera: 1.22 minutos
Trabajo procesado en t=12.213601665435876, tiempo de espera: 1.38 minutos
Trabajo procesado en t=12.42214727166483, tiempo de espera: 1.85 minutos
Trabajo procesado en t=13.342235945703926, tiempo de espera: 1.49 minutos
Trabajo procesado en t=13.792030173184697, tiem